<div style="display: flex; align-items: center;">
  <img src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/DecisionIntelligenceSemanticKernelLogo.png" width="60px" style="margin-right: 10px;">
  <span style="font-size: 1.5em; font-weight: bold;">Semantic Kernel - Decisions with Multi-AI Agent Personas</span>
</div>

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.8"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.8"

//using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:ReasoningEndpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:ReasoningAPIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ReasoningModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.8 Microsoft.Extensions.Configuration.Json, 9.0.8

In [2]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.62.0" 
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.62.0"
#r "nuget: Microsoft.SemanticKernel.Agents.Abstractions, 1.62.0"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: azureOpenAIModelDeploymentName!,
        endpoint: azureOpenAIEndpoint!,
        apiKey: azureOpenAIAPIKey!)
    .Build();

Installed Packages Microsoft.SemanticKernel, 1.62.0 Microsoft.SemanticKernel.Agents.Abstractions, 1.62.0 Microsoft.SemanticKernel.Agents.Core, 1.62.0

### Step 2 - Decisions with Multi-Agent Personas 

> "The many are smarter than the few when it comes to solving problems, predicting outcomes, and innovating solutions, as long as there is diversity of opinion, independence, and decentralization."  
>
> -- <cite>James Surowiecki (Author "The Wisdom of Crowds")</cite> 

Single agent personas are effective constructs in Semantic Kernel, but can largely be replicated in other mechanisms with prompts. **Creating multiple personas (experts) that can interact with each other is the true power of the Semantic Kernel Agent functionality. This allows multiple chat agents to interact in various ways.** 

Execute the cell below to illustrate how two different personas can interact in a decision-making process moderated by a decision intelligence expert that makes the final reasoning conclusiion on the decision task at hand. In the output notice key steps:
* Each expert (Dwight and Ben) make their recommendation, based on their decision persona
* The final recommendation is made based on the synthesis of the agent recommendations  
* This multi-agent interaction to converge on a single decision mimic the self-consistency GenAI pattern 

Note: The execution below can take the some time. Use GPT-5-mini to optimize speed of the execution.  

In [3]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.ChatCompletion;
using System.Threading;

#pragma warning disable SKEXP0110
// Create a new chat TerminationStrategy that will terminate when the final message contains the term "final decision"
private sealed class ApprovalTerminationStrategy : TerminationStrategy
{
    // Terminate when the final message contains the term "approve"
    protected override Task<bool> ShouldAgentTerminateAsync(Agent agent, IReadOnlyList<ChatMessageContent> history, CancellationToken cancellationToken)
        => Task.FromResult(history[history.Count - 1].Content?.Contains("final decision", StringComparison.OrdinalIgnoreCase) ?? false);
}

var assistantDwightEisenhower = new ChatCompletionAgent
{
    Kernel = semanticKernel,
    Name = "DwightEisenhower", // Dwight Eisenhower persona
    Description = "A decision intelligence agent that uses the Eisenhower Decision Priorotization Matrix.",
    Instructions = "You are a decision intelligence strategist with the knowledge of Dwight Eisenhower's Decision Prioritization Matrix. Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. Provide structured, logical, and comprehensive advice."
};

var assistantBenFranklin = new ChatCompletionAgent
{
    Kernel = semanticKernel,
    Name = "BenFranklin", // Ben Franklin persona
    Description = "A decision intelligence agent that uses the Ben Franklin Decision Framework.",
    Instructions = "You are a decision intelligence strategist with the knowledge of Ben Franklin's Decision Framework. Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. Provide structured, logical, and comprehensive advice."
};

var assistantDecisionMaker = new ChatCompletionAgent
{
    Kernel = semanticKernel,
    Name = "DecisionMaker",
    Description = "A decision intelligence agent that approves the final decision.",
    Instructions = """
    You are a Decision Intelligence maker. Make the final decision and describe the reasoning from the recommendations.
    You're laser focused on the goal at hand. Don't waste time with chit chat.
    Consider suggestions when refining an idea. 
    When ready to approve the final the decision, use the words final decision to approve the final decision.
    """
};

// Create a chat for agent interaction.
AgentGroupChat chat =
    new(assistantDwightEisenhower, assistantBenFranklin, assistantDecisionMaker)
    {
        ExecutionSettings =
            new()
            {
                // Here a TerminationStrategy subclass is used that will terminate when
                // an assistant message contains the term "approve".
                TerminationStrategy =
                    new ApprovalTerminationStrategy()
                    {
                        // Only the art-director may approve.
                        Agents = [assistantDecisionMaker],
                        // Limit total number of turns
                        MaximumIterations = 6,
                    }
            }
    };

// Invoke chat and display messages.
ChatMessageContent decisionRecommendation = new(AuthorRole.User, """
### Decision To Process:  
I am thinking of buying a rental property versus investing in Bitcoin. 
What things should I consider in making this decision?
### Decision Constraints:  
1) You want to optimize for long-term financial growth, but also mitigate risk. 
2) You have a budget of $500,000 cash for the investment. 
3) You are the in early 40s and the breadwinner of your family and need to ensure financial stability.
4) You don't have much debt, but have kids going to college in the next 5 years.
5) You are not an expert in real estate or cryptocurrency, but have a basic understanding of both.

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""");
chat.AddChatMessage(decisionRecommendation);
decisionRecommendation.Content.DisplayAs("text/markdown");
Console.WriteLine();
Console.WriteLine();

await foreach (ChatMessageContent response in chat.InvokeAsync())
{
    response.Content.DisplayAs("text/markdown");
}

### Decision To Process:  
I am thinking of buying a rental property versus investing in Bitcoin. 
What things should I consider in making this decision?
### Decision Constraints:  
1) You want to optimize for long-term financial growth, but also mitigate risk. 
2) You have a budget of $500,000 cash for the investment. 
3) You are the in early 40s and the breadwinner of your family and need to ensure financial stability.
4) You don't have much debt, but have kids going to college in the next 5 years.
5) You are not an expert in real estate or cryptocurrency, but have a basic understanding of both.

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.

| Factor                                 | Rental Property                                                                                      | Bitcoin                                                                                               |
|----------------------------------------|------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------|
| Capital & Financing Requirements       | ~\$100K-125K minimum down per property; rest financed; leverage spreads risk across multiple assets  | \$500K upfront; fractional purchases possible; leverage via margin increases risk of margin calls    |
| Cash Flow & Income Stability           | Predictable rent income; covers mortgage & expenses; provides inflation hedge                        | No periodic income; purely reliant on price appreciation; optional staking yields but adds complexity |
| Long-Term Return Potential             | Historically 6-12% net annual returns; benefits from leverage & tax incentives                       | High historical returns but volatile; potential for outsized gains; long-term trajectory uncertain    |
| Volatility & Risk Profile              | Moderate volatility; local market downturns; vacancy & tenant risk                                   | Very high volatility; subject to market sentiment swings; regulatory & technological risks            |
| Liquidity & Exit Strategy              | 3-6 months typical sale timeline; transaction costs; potential illiquidity during downturns          | 24/7 global markets; near-instant liquidation; low transaction friction                                |
| Expertise & Management Effort          | Requires landlord skills or property manager; maintenance & tenant relations                         | Minimal operational effort; need secure custody & understanding of wallet security                    |
| Tax Implications                       | Depreciation & mortgage interest deductions; long-term capital gains; 1031 exchanges                | Taxable events on trades; long-term vs short-term capital gains; evolving crypto tax compliance       |
| Impact on College Fund & Cash Needs    | Generates ongoing cash flow; can tap equity via HELOC; slower principal access                       | Quick liquidation for cash; timing risk; may force sale at loss if market down                        |
| Diversification & Correlation          | Low correlation with equities; geographic concentration risk                                         | Uncorrelated with most asset classes; correlation spikes in risk-off events                           |
| Leverage & Risk Mitigation             | Mortgage leverage boosts returns; interest rate and refinancing risk                                 | Margin trading available but increases potential for forced liquidation                              |
| Regulatory & Market Risks              | Subject to zoning, landlord-tenant laws, tax code changes                                            | Regulatory uncertainty; potential for bans/restrictions; network security & protocol risks            |
| Time Horizon & Suitability             | Best for medium to long term (5+ years); must weather real estate cycles                             | Suitable as a smaller, high-risk allocation; high timing sensitivity; recommended longer hold period  |

| Consideration                              | Rental Property                                                                                       | Bitcoin                                                                                                 |
|--------------------------------------------|-------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------|
| Initial Capital Deployment                 | Requires ~20–25% down ($100K–125K) on a $500K property; mortgage covers remainder                       | Deploy full $500K; fractional purchases; no mortgage needed                                             |
| Income Generation                          | Steady monthly rent covers mortgage, expenses; surplus can fund college or savings                     | No periodic income; relies on price appreciation; optional staking yields but less predictable          |
| Long-Term Appreciation                     | Historical home price growth ~3–5% annually + rent increases; inflation hedge                          | High historical returns but extremely volatile; future trajectory uncertain                              |
| Risk Profile                               | Moderate: vacancy, tenant default, property damage, local market downturns                            | High: price swings, regulatory changes, security breaches, technological risks                           |
| Liquidity & Exit Timing                    | 3-6 months to sell; transaction costs ~5–10%; illiquid in downturns                                   | 24/7 global markets; near-instant trades; potential exchange outages or liquidity squeezes               |
| Management & Expertise                     | Landlord duties or property manager fees; legal compliance; maintenance                               | Minimal operations; must secure wallets/keys; guard against hacks and phishing                          |
| Tax & Regulatory                            | Mortgage interest & depreciation deductions; long-term capital gains; 1031 exchange options            | Capital gains taxed on trade; evolving crypto regulations; rigorous record-keeping required              |
| Leverage & Financing                       | Mortgage leverage boosts returns; interest rate and refinancing risk                                  | Margin trading possible but high liquidation risk; generally unleveraged unless taking on extra risk    |
| Family Stability & College Funding         | Predictable cash flow supports tuition; can tap equity via HELOC for emergencies                       | Liquidation risk if sold during price dips; timing market to meet tuition deadlines adds stress         |
| Diversification & Correlation              | Low correlation with equities; geographic concentration risk                                          | Uncorrelated asset class; correlation spikes in broader market sell-offs                                 |
| Time Horizon Suitability                   | Best for 5+ year horizon to smooth cycles and cover transaction costs                                 | More speculative; if allocated, treat as smaller position with long-term holding expectations           |

| Consideration                      | Rental Property                                                                                         | Bitcoin                                                                                                  |
|------------------------------------|----------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------|
| Capital deployment                 | 20–25% down on a \$500K property (~\$100–125K); finance remainder; preserves \$375–400K reserves          | Deploy full \$500K; fractional purchases; no financing; ties up entire cash                              |
| Expected return                    | 6–12% net annual (rent + appreciation); stable growth with leverage and tax benefits                      | High historical returns but extremely volatile; no guaranteed upside                                     |
| Cash flow                          | Predictable monthly rent to cover mortgage, expenses and contribute to college fund                       | No periodic income; optional staking yields (~4–10%) with added complexity                                |
| Risk profile                       | Moderate: vacancy, tenant issues, local market cycles; can mitigate with insurance & property manager     | High: large price swings, regulatory changes, custody/security risks                                     |
| Liquidity & exit                   | 3–6 months to sell; ~5–10% transaction costs; limited in downturns                                       | Near-instant global trading; low fees; risk of exchange outages                                          |
| Management & expertise             | Requires landlord skills or property manager fees; maintenance oversight                                | Minimal operations; requires secure wallet management and diligent key storage                          |
| Tax & regulatory                   | Mortgage interest and depreciation deductions; stable tax code; 1031 exchange options                   | Taxable events on trades; evolving crypto regulations; meticulous record-keeping                         |
| Impact on college funding          | Steady cash flow; can tap equity via HELOC; slower principal access                                      | Quick liquidation possible but timing risk may force sale at loss if market dips                         |
| Diversification & correlation      | Low correlation with equities; localized geographic risk                                                 | Uncorrelated with traditional assets; correlation spikes during market stress                            |
| Time horizon                       | Best over 5+ years to weather cycles and cover transaction costs                                         | Suitable as a small, high-risk allocation for 5+ years                                                   |
| Final Decision                     | Allocate ~\$450K to purchase one or two rental properties for stable income, diversification and growth (final decision) | Allocate ~\$50K (10%) to Bitcoin for upside potential and portfolio diversification                      |

### Step 3 - Decisions with Advanced Multi-Agent Decision Orchestration  

The previous example was quite simplistic as it basically put several agents together with a moderator and it completed once the words "Final Decision" were emitted. Advanced multi-agent orchestrations can provide much more powerful control around how agents work together. Semantic Kernel supports several orchestration patterns, each designed for different collaboration scenarios. These patterns are available as part of the framework and can be easily extended or customized. 

One of these patterns is the Magnetic orchestration, which is a powerful multi-agent pattern designed for managing complex, open-ended tasks. The Magentic orchestration in Semantic Kernel is inspired by the "Magentic-One" system. It involves a dedicated Magentic manager that coordinates a team of specialized agents to solve intricate problems. This manager is powered by an AI model to perform the coordination. It selects the next agent based on different factors in the process like: task progress, agent capabilities, evolving context etc. 

In [4]:
// Import the new Magentic orchestration package and Semantic Kernel InProcessRuntime
#r "nuget: Microsoft.SemanticKernel.Agents.Magentic, 1.62.0-preview" 
#r "nuget: Microsoft.SemanticKernel.Agents.Orchestration, 1.62.0-preview" 
#r "nuget: Microsoft.SemanticKernel.Agents.Runtime.InProcess, 1.62.0-preview" 

using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Magentic;
using Microsoft.SemanticKernel.Agents.Orchestration;
using Microsoft.SemanticKernel.Agents.Runtime.InProcess;

// Create a Devil's advocate agent that rigorously tests and challenges recommendations
var assistantDevilsAdvocate = new ChatCompletionAgent
{
    Kernel = semanticKernel,
    Name = "DevilsAdvocate",
    Description = "A decision intelligence agent that acts as a devil's advocate, rigorously testing and challenging recommendations.",
    Instructions = @"
You are a devil's advocate decision intelligence strategist. 
Your job is to critically evaluate any proposed recommendations or decisions by:
  1. Identifying hidden assumptions and blind spots  
  2. Raising potential risks, unintended consequences, and edge cases  
  3. Questioning feasibility, resource constraints, and stakeholder buy-in  
  4. Suggesting alternative viewpoints or counter-arguments  
  5. Prompting further data, evidence, or analysis where needed  
Respond in a structured way: state each challenge clearly, explain why it matters, 
and recommend how to address or investigate it further."
};


#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001
var magenticManager = new StandardMagenticManager(
    semanticKernel.GetRequiredService<IChatCompletionService>(),
    new OpenAIPromptExecutionSettings())
{
    MaximumInvocationCount = 4, // Maximum number of invocations per orchestration
};


// Initalize the ChatHistory to store the messages exchanged during the orchestration
ChatHistory history = [];

// Define a response callback to handle the responses from the orchestration
// All the agent messages will be added in the history
ValueTask responseCallback(ChatMessageContent response)
{
    history.Add(response);
    return ValueTask.CompletedTask;
}

// Create the MagenticOrchestration with the agents and the response callback
MagenticOrchestration orchestration = new MagenticOrchestration(
    magenticManager,
    assistantDevilsAdvocate,
    assistantDwightEisenhower,
    assistantBenFranklin,
    assistantDecisionMaker)
{
    ResponseCallback = responseCallback,
};

// Create an Semantic Kernel InProcessRuntime to run the orchestration
InProcessRuntime runtime = new InProcessRuntime();
await runtime.StartAsync();

string decisionToProcess = """
### Decision To Process:  
I am thinking of buying a rental property versus investing in Bitcoin. 
What things should I consider in making this decision?
### Decision Constraints:  
1) You want to optimize for long-term financial growth, but also mitigate risk. 
2) You have a budget of $500,000 cash for the investment. 
3) You are the in early 40s and the breadwinner of your family and need to ensure financial stability.
4) You don't have much debt, but have kids going to college in the next 5 years.
5) You are not an expert in real estate or cryptocurrency, but have a basic understanding of both.

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";
decisionToProcess.DisplayAs("text/markdown");
Console.WriteLine("----------------------------------------------------------");
Console.WriteLine("----------------------------------------------------------");
"### FINAL RECOMMENDATION".DisplayAs("text/markdown");

// Invoke the orchestration with the decision to process
var orchestrationResult = await orchestration.InvokeAsync(decisionToProcess, runtime);
// Display the decision recommendation from the orchestration result with a timeout of 300 seconds
string decisionRecommendation = await orchestrationResult.GetValueAsync(TimeSpan.FromSeconds(300));
decisionRecommendation.DisplayAs("text/markdown");

Console.WriteLine("----------------------------------------------------------");
Console.WriteLine("----------------------------------------------------------");
"### ORCHESTRATION HISTORY".DisplayAs("text/markdown");

// Show the history of messages exchanged during the orchestration
foreach (ChatMessageContent message in history)
{
    var messageString = $"""
    ### {message.Role} - {message.AuthorName}: 
    {message.Content}
    """;
    messageString.DisplayAs("text/markdown");
}

// Wait for the runtime to complete all tasks
// Dispose the runtime to clean up resources
await runtime.RunUntilIdleAsync();
await runtime.DisposeAsync()

Installed Packages Microsoft.SemanticKernel.Agents.Magentic, 1.62.0-preview Microsoft.SemanticKernel.Agents.Orchestration, 1.62.0-preview Microsoft.SemanticKernel.Agents.Runtime.InProcess, 1.62.0-preview

### Decision To Process:  
I am thinking of buying a rental property versus investing in Bitcoin. 
What things should I consider in making this decision?
### Decision Constraints:  
1) You want to optimize for long-term financial growth, but also mitigate risk. 
2) You have a budget of $500,000 cash for the investment. 
3) You are the in early 40s and the breadwinner of your family and need to ensure financial stability.
4) You don't have much debt, but have kids going to college in the next 5 years.
5) You are not an expert in real estate or cryptocurrency, but have a basic understanding of both.

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.

----------------------------------------------------------
----------------------------------------------------------


### FINAL RECOMMENDATION

| Factor                     | Weight | Rental Rating (1–5) | Rental Weighted | Bitcoin Rating (1–5) | Bitcoin Weighted |
|----------------------------|-------:|---------------------:|----------------:|----------------------:|-----------------:|
| Risk (volatility)          |   0.25 |                    4 |           1.00  |                     2 |            0.50  |
| Liquidity                  |   0.20 |                    2 |           0.40  |                     5 |            1.00  |
| Cash Flow Stability        |   0.20 |                    5 |           1.00  |                     2 |            0.40  |
| Growth Potential           |   0.15 |                    3 |           0.45  |                     5 |            0.75  |
| Tax & College Funding Fit  |   0.10 |                    4 |           0.40  |                     2 |            0.20  |
| Expertise & Time Commitment|   0.10 |                    2 |           0.20  |                     4 |            0.40  |
| **Total Score**            |   1.00 |                      |           3.45  |                        |            3.25  |

----------------------------------------------------------
----------------------------------------------------------


### ORCHESTRATION HISTORY

### Assistant - DevilsAdvocate: 
| Challenge                                                         | Why It Matters                                                                                                                               | How to Address or Investigate                                                                                                                   |
|-------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------|
| Assumption of stable rental income                                | Rental markets can fluctuate with local employment trends, interest rates, and regulation; overestimating stability may harm cash flow       | Analyze historical vacancy rates, rent growth, and landlord regulations in targeted areas; stress‐test income under downturn scenarios            |
| Assumption of Bitcoin’s long-term appreciation                    | Crypto markets are highly volatile and influenced by regulatory moves, tech adoption, and market sentiment; past rallies don’t guarantee future gains | Review scenario analyses for crypto under various regulatory and adoption outcomes; monitor on‐chain metrics and institutional flows              |
| Liquidity differences                                              | Real estate is illiquid and transaction costs are high; Bitcoin can be liquidated quickly but may face market depth and tax event issues    | Decide on required liquidity buffers for emergencies and college funding; plan exit strategies and timelines for both asset types                |
| Management burden and expertise gap                                | Property requires time, skills for tenant screening, maintenance, and legal compliance; crypto needs security management and tax reporting    | Factor in property management fees or partner with agents; implement best practices for digital asset custody (cold storage, audits, insurance) |
| Large concentrated investment risk                                 | Allocating $500K to one property or a large crypto position increases exposure to idiosyncratic risk                                           | Consider diversification across multiple properties or a crypto basket; evaluate smaller initial allocations with phased scaling                |
| Upcoming college funding needs                                     | Tying too much capital in volatile assets or illiquid real estate may conflict with obligations in 5 years                                     | Project cash flow needs for tuition; maintain a separate college fund or short-term bonds/CDs to ensure availability                              |
| Tax and regulatory complexity                                      | Rental income, capital gains, and crypto transactions have different tax treatments and reporting burdens                                      | Consult a tax advisor to compare depreciation, 1031 exchanges, crypto wash-sale rules, and potential state/sales tax impacts                     |
| Family risk tolerance and stakeholder buy-in                       | As breadwinner, you need family support for major risk decisions; misalignment can cause stress and conflict                                   | Discuss scenarios with spouse/financial planner; set clear risk limits and contingency plans; use visual tools to illustrate best/worst cases    |

### Assistant - DwightEisenhower: 
| Factor                    | Urgency & Importance Quadrant | Immediate Attention? |
|---------------------------|-------------------------------|----------------------|
| Liquidity                 | Urgent & Important (Q1)       | Yes                  |
| College funding timeline  | Urgent & Important (Q1)       | Yes                  |
| Expertise requirements    | Urgent & Important (Q1)       | Yes                  |
| Cash flow                 | Not Urgent & Important (Q2)   | No                   |
| Tax advantages            | Not Urgent & Important (Q2)   | No                   |
| Growth potential          | Not Urgent & Important (Q2)   | No                   |
| Risk                      | Not Urgent & Important (Q2)   | No                   |
| Time commitment           | Not Urgent & Important (Q2)   | No                   |

### Assistant - BenFranklin: 
| Factor                     | Weight | Rental Rating (1–5) | Rental Weighted | Bitcoin Rating (1–5) | Bitcoin Weighted |
|----------------------------|-------:|---------------------:|----------------:|----------------------:|-----------------:|
| Risk (volatility)          |   0.25 |                    4 |           1.00  |                     2 |            0.50  |
| Liquidity                  |   0.20 |                    2 |           0.40  |                     5 |            1.00  |
| Cash Flow Stability        |   0.20 |                    5 |           1.00  |                     2 |            0.40  |
| Growth Potential           |   0.15 |                    3 |           0.45  |                     5 |            0.75  |
| Tax & College Funding Fit  |   0.10 |                    4 |           0.40  |                     2 |            0.20  |
| Expertise & Time Commitment|   0.10 |                    2 |           0.20  |                     4 |            0.40  |
| **Total Score**            |   1.00 |                      |           3.45  |                        |            3.25  |